In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [3]:
def Combainer(disease, day_before=False):
    Data_Fibro = Data[Data['trackable_name']== disease].drop_duplicates(('user_id','checkin_date'))
    list_of_user_id = Data_Fibro["user_id"].values; list_of_checkin = Data_Fibro["checkin_date"].values
    
    list_of_df = []; i=0
    
    for pair in log_progress(zip(list_of_user_id,list_of_checkin), every=1,size=20544):
        df = Data[(Data['user_id'] == pair[0]) & (Data['checkin_date'] == pair[1])]
        
        list_of_df.append(df);# i+=1 
        
        #print("{0} % is done!".format( round(i/20544,3) *100))
    return list_of_df

## Import with parse_dates 

In [4]:
Data = pd.read_csv("fd-export.csv",parse_dates=[4], low_memory=False)

## Change user_id

In [5]:
## const - number of unique users
const = np.unique(Data.user_id)
## List of all users in N names
list_of_user = [i for i in range(1, len(const)+1)]
## Dictionary d["old_user_id"]= N_name
Dict = dict(zip(const, list_of_user))
## List of names
List_of_int_names = [ Dict[i] for i in Data.user_id]
## Changing user_id
Data['user_id']=List_of_int_names

## Delete NaN in trackable_value 

In [6]:
Data['trackable_value'].fillna(value="-",inplace=True)

## Drop country and trackable_id 

In [7]:
Data.drop(['country',"trackable_id"],axis=1, inplace=True)

In [8]:
Data.dropna(axis=0, how='any',inplace=True)

## Датасет 1
Выделяем болезнь из списка. Для всех случаев фиксации болезни в базе делаем отдельную таблицу с данными за этот день и прошлый. Потом эти таблицы переделываем в один формат и соединяем

## 20672 rows

In [10]:
List_of_df = Combainer("Fibromyalgia")

A Jupyter Widget

KeyboardInterrupt: 

In [9]:
def dict_feat(df):
    ## Columns
    feat = Fin_base.columns
    #Инициализация рядка с Not mentioned значениями
    row=['Not Mentioned']*len(feat)
    #Add Age
    row[0]=df['age'].unique()[0]
    #Add sex
    row[1]=df['sex'].unique()[0]
    
    ##Add Season
    
    inter = list(enumerate(feat))[2:]
    for ind,name in inter:
        df_temp = df[df['trackable_name'] == name]
        if df_temp.empty:
            continue
        else:
            value = df_temp['trackable_value'].values[0]
            row[ind] = value
    return row

In [12]:
Fin_base= pd.DataFrame(columns = ['age',"sex",'Fibromyalgia', 'pressure', 'temperature_max', 'humidity',
                    'precip_intensity', 'temperature_min', 'icon', 'Anxiety', 'Depression',
                    'Fatigue', 'Headache', 'Migraine', 'Nausea', 'Brain fog', 'tired',
                    'Neck pain', 'Joint pain', 'Back pain', 'Diarrhea', 'Dizziness',
                    'Insomnia'])
for df in log_progress(List_of_df,every= 1 ,size=20544):
    Fin_base.loc[len(Fin_base)] = dict_feat(df)
Fin_base

A Jupyter Widget

,age,sex,Fibromyalgia,pressure,temperature_max,humidity,precip_intensity,temperature_min,icon,Anxiety,...,Migraine,Nausea,Brain fog,tired,Neck pain,Joint pain,Back pain,Diarrhea,Dizziness,Insomnia
0,23.0,female,2,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,1,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
1,31.0,female,1,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,0,Not Mentioned,Not Mentioned,-,Not Mentioned,Not Mentioned,2,Not Mentioned,Not Mentioned,2
2,31.0,female,1,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,0,Not Mentioned,Not Mentioned,-,Not Mentioned,Not Mentioned,2,Not Mentioned,Not Mentioned,1
3,40.0,male,2,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
4,32.0,male,2,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
5,31.0,female,3,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,0,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,3
6,27.0,doesnt_say,2,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,1,Not Mentioned
7,48.0,female,3,1023.0,70.0,84.0,0.0011,52.0,partly-cloudy-day,Not Mentioned,...,Not Mentioned,Not Mentioned,Not Mentioned,-,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
8,27.0,female,2,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,Not Mentioned,Not Mentioned,1,Not Mentioned,Not Mentioned,Not Mentioned,2,Not Mentioned,Not Mentioned,Not Mentioned
9,27.0,female,1,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,...,Not Mentioned,Not Mentioned,1,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned


In [14]:
Fin_base.to_csv('Fibro_base.csv')